In [1]:
import sys 
sys.path.append("..") 
from pubmedkit.load_pubmed_file import load_pubmed_file
import pandas as pd

In [2]:
records = load_pubmed_file("../data/pubmed_Arabidopsis_ChIP.txt",output_type='dict')

In [3]:
# records

In [4]:
from langchain_community.llms.ollama import Ollama
from langchain_core.prompts import PromptTemplate
from tqdm import tqdm

In [5]:
planttemplate = """ 

Transcription factor (TF)  is a protein that controls the rate of transcription of genetic information from DNA to messenger RNA, by binding to a specific DNA sequence.   Chromatin immunoprecipitation (ChIP) is a type of immunoprecipitation experimental technique used to investigate the interaction between proteins and DNA in the cell. It aims to determine whether specific proteins are associated with specific genomic regions, such as transcription factors on promoters or other DNA binding sites, and possibly define cistromes. Arabidopsis or Arabidopsis thaliana is model organism for studying the function of transcription factors. Arabidopsis thaliana is not name of the TF.

Context:{context}.

Question: If the above content related to research in transcription factor of Arabidopsis, please give name of Transcription factor. If don't find name of TF, just say that you don't know or don't find TF's name, don't try to make up an answer.

"""


In [6]:
llm = Ollama(
    model= 'llama3.1',
    base_url='http://localhost:11434',  # 可以填 192.168.127.87
    temperature=0
)

In [7]:

for pmid in records:
    abstract = records[pmid]['abstract']
    prompt = PromptTemplate(input_variables=['text'], template=planttemplate)
    chain = prompt | llm
    response = chain.invoke(abstract).rstrip('\n')
    print(records[pmid]['pmid'],response)
    records[pmid]['plant_response'] = response

25215497 No.
30909785 TGA9
36403487 WRKY29
32445409 Yes. ABI3.
37382898 BBX14
26817596 Yes. I don't know.
35806432 Yes. AtOFP1.
23340744 ORE1 (ANAC092)
35968123 CCA1
32279407 AINTEGUMENTA (ANT) and AINTEGUMENTA-LIKE 6 (AIL6)/PLETHORA 3 (PLT3)
33951433 PIF7
37345955 KANADI 1, MYB DOMAIN PROTEIN 44, and PHYTOCHROME INTERACTING FACTOR 4.
32005383 BZR1
23404561 Di19
36142886 GmbZIP152
30374363 EsNAC1
38075399 Yes. AGL15.
38837414 WRKY22
28529706 Yes.
29525962 Yes.
26252246 WRKY46
24164091 HAT1 and HAT3.
34594347 GmWRKY46
26010075 Yes.
30690513 ANAC032
33206180 BES1/BZR1
39279854 Yes.
34850203 AGL18
36534453 No.
27297966 DOF transcription factor OBP4.
33656670 Yes. I don't know.
33719309 NAC075.
21586587 Yes. Arabidopsis thaliana.
28782043 Yes.
38418388 WRKY6
36519581 ANAC087
38861819 Yes. WRKY47.
25479036 Yes. PdhR.
35755678 Yes. FIT.
24009617 Yes. NAC and MYB.
31776249 Upstream Regulator of IRT1 (URI)
30445619 Yes. I don't know.
23951554 ATAF1
30002259 GNC and GLK2.
36909399 MeGI
35955798

KeyboardInterrupt: 

In [8]:
records_pd = pd.DataFrame.from_dict(records).T

In [9]:
records_pd.to_excel('pubmed_ath_llm.xlsx')